In [ ]:
#entra: el excel con diferentes anotaciones 
#sale: ejecucion de las imagenes ya cortadas, en la misma carpeta en la que esta este jupyternotebbok

In [ ]:
import numpy as np
import cv2 as cv2
import pandas as pd
from tqdm import tqdm
import xml.etree.ElementTree as ET
import os
from subprocess import call

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import xml.etree.ElementTree as ET
def transformacion_ndpi2pixeles(anotaciones_ndpi):
    anotaciones_pixeles=[]
    for anotacion in anotaciones_ndpi:
        list_aux=[]
        x_ndpi = anotacion[0]
        y_ndpi = anotacion[1]
        
        x_offset = anotacion[3]
        y_offset = anotacion[4]
        
        mpp_x = anotacion[5]
        mpp_y = anotacion[6]
        
        width_l0 = anotacion[7]
        height_l0 = anotacion[8]
        
        x_pixeles = x_ndpi-x_offset
        y_pixeles = y_ndpi-y_offset
        
        x_pixeles /=1000
        y_pixeles /=1000
        
        x_pixeles /= mpp_x
        y_pixeles /= mpp_y
       
        x_pixeles += width_l0/2
        y_pixeles += height_l0/2
        
        #se aplica el ESCALAMIENTO de 40x a 10x
        coef_escalmiento=10/40
        x_pixeles *= coef_escalmiento 
        y_pixeles *= coef_escalmiento
        
        
        #como el radio es escalar solo escala, no se traslada
        radius = anotacion[2]
        mean_mpp=(mpp_x+mpp_y)/2
        radio_pixeles =radius/(1000*mean_mpp)#forzamos que sea int porque son pixeles
        radio_pixeles *= coef_escalmiento
        list_aux.append(int(x_pixeles))
        list_aux.append(int(y_pixeles))
        list_aux.append(int(radio_pixeles))
        anotaciones_pixeles.append(list_aux)
        #print('x_pix=',list_aux[0],'y_pix=',list_aux[1],'radio_pix=',list_aux[2])
    return anotaciones_pixeles

def get_esquinas_parches(lado_parche, anotaciones_pixeles): 
    contenedor_esquinas=[]
    for anotacion in anotaciones_pixeles:
        x_roi = anotacion[0]
        y_roi = anotacion[1]
        radio_roi = anotacion[2]
        ppr = int(radio_roi/lado_parche)
        
        esquinas=[]#son las esquinas para la anotacion o roi especifica
        ### para generar las esquinas superiores ###    
        #### para generar la primera fila de la parte superior ####
        y_p_1 = y_roi - ppr*lado_parche
        xs_lista = get_x_coord_sympy(y_p_1, x_roi, y_roi, radio_roi)
        x_p_menor=int(min(xs_lista))
        x_p_mayor=int(max(xs_lista))
        esquinas_fila = get_esquinas_fila(y_p_1, x_p_menor, x_p_mayor, lado_parche)
        esquinas.append(esquinas_fila)
        #### para generar de la 2º fila hacia abajo ####
        for i in range(1, ppr):
            y_p_izq = y_p_1 + i*lado_parche#bajo a la siguiente fila
            xs_lista = get_x_coord_sympy(y_p_izq, x_roi, y_roi, radio_roi)
            x_p_menor=int(min(xs_lista))
            x_p_mayor=int(max(xs_lista))
            esquinas_fila = get_esquinas_fila(y_p_izq, x_p_menor, x_p_mayor, lado_parche)
            esquinas.append(esquinas_fila)
        ### para generar las esquinas inferiores ###    
        esquinas_inferiores=[]
        for lista in esquinas:
            fila=[]
            for esquina in lista:
                x_1 = esquina[0]
                y_1 = esquina[1]
                punto_simetrico = pto_simetrico(x_1,y_1,x_roi,y_roi)
                #genera pto simetrico c/r al centro del circulo roi#
                #hay que restarle lado parche a cada pto generado para no se salgan del roi#
                #y con esto tener una sola funcion de draw#
                fila.append([punto_simetrico[0]-lado_parche, punto_simetrico[1]-lado_parche])
            fila.reverse()
            esquinas_inferiores.append(fila)
        esquinas_inferiores.reverse()
        #no se pueden juntar dentro del for anterioir porque genera un ciclo infinito 
        ### para juntar esquinas superiores con inferiores ###
        for aux in esquinas_inferiores:
            esquinas.append(aux)
        contenedor_esquinas.append(esquinas)
    return contenedor_esquinas

#---------------------------------------------------------------------------------------------------------------
def pto_simetrico(x_1, y_1, x_roi, y_roi):
    x_2=2*x_roi-x_1
    y_2=2*y_roi-y_1
    return [x_2,y_2]

def get_esquinas_fila(y_p_2,x_p_1_menor,x_p_1_mayor, lado_parche):
    esquinas_fila=[]
    for x in range(x_p_1_menor,x_p_1_mayor-lado_parche, lado_parche):
        esquina=[]
        esquina.append(x)
        esquina.append(y_p_2)
        esquinas_fila.append(esquina)
    return esquinas_fila#es una lista de listas, cada lista interna es un punto

from sympy.solvers import solve
from sympy import Symbol

def get_x_coord_sympy(y_p,x_roi,y_roi,radio_roi):
    x = Symbol('x')
    x_soluciones = solve( (x-x_roi)**2 + (y_p-y_roi)**2 - radio_roi**2,x)
    return x_soluciones

def contador_parches(esquinas):
    count = 0
    for fila in esquinas:
        for esquina in fila:
            count+=1
    return count

def abrebiar_clases(clase):
    abreviacion_clase={'No tumor':'nt',#tejido no canceroso
                        'Negativo':'sr',# 'Negativo':'sin reactividad'     #tejido canceroso
                        'Positivo no lineal':'rnl',                        #tejido canceroso
                        'Positivo lineal casi imperceptible':'rlci',       #tejido canceroso
                        'Positivo lineal débil':'rld',                     #tejido canceroso
                        'Positivo lineal fuerte':'rlf',                    #tejido canceroso
                        }
    return abreviacion_clase[clase]
#---------------------------------------------------------------------------------------------------------------
def get_comandos_terminal(contenedor_esquinas_parche,lado_parche,path_imagen_ndpi,id_anotacion,clase_anotacion):
    comandos=[]
    abrev_clase=str(abrebiar_clases(clase_anotacion))
    for esquinas in contenedor_esquinas:
        ID_fila = 0
        for fila in esquinas:
            ID_col = 0
            for esquina in fila:
                x_sup_izq = str(int(esquina[0]))
                y_sup_izq = str(int(esquina[1]))
                ancho_parche = str(int(lado_parche))
                largo_parche = str(int(lado_parche))
                ID_parche = 'a'+str(id_anotacion) + 'f'+str(ID_fila) + 'c'+str(ID_col) + '_clas_' + abrev_clase                
                comandos.append('ndpisplit -Ex10,z0,'+ x_sup_izq +','+ y_sup_izq +','+ ancho_parche +','+ largo_parche +',idparche_'+ ID_parche +' -K '+path_imagen_ndpi)
                #print('ndpisplit -Ex40,z0,0,0,'+str(int(esquina[0]))+','+str(int(esquina[1]))+',mylabel -K ejemplo.ndpi')
                ID_col +=1
            ID_fila +=1
    return comandos


def save_comandos_txt(comandos_ndpi):#en la carpeta actual crea este archivo
    with open('comandos_terminal.txt', "a") as file:#abrirlo en modo 'a' es para escribirle cosas al final al archivo
        for comando in comandos_ndpi:
            file.write(str(comando) +"\n")
        file.close()

In [ ]:
def get_esquinas_parches_v2(lado_parche, anotaciones_pixeles):
    contenedor_esquinas=[]
    for anotacion in anotaciones_pixeles:
        x_roi = anotacion[0]
        y_roi = anotacion[1]
        radio_roi = anotacion[2]
        esquinas=[]#son las esquinas para la anotacion o roi especifica
        x_sup_izq , y_sup_izq= x_roi-radio_roi, y_roi-radio_roi
        x_inf_der, y_inf_der = x_roi+radio_roi, y_roi+radio_roi
        for j in range(y_sup_izq, y_inf_der, lado_parche):
            esquinas_fila=[]
            for i in range(x_sup_izq,x_inf_der, lado_parche):
                esquinas_fila.append([i,j])
            esquinas.append(esquinas_fila)
        contenedor_esquinas.append(esquinas)
    return contenedor_esquinas

# MAIN

In [ ]:
path_excel_infoHER2_luis='her2_luis.xlsx'
anotaciones_excel = pd.read_excel(path_excel_infoHER2_luis)


lado_parche=299

#lo usaremos cuanod se use el filtro, para las imagenes nuevas
path_carpeta_base='/home/luise/Desktop/memoria/'

for index, anotacion in tqdm(anotaciones_excel.iterrows()):
    anotacion_pixeles = [[int(anotacion['x_pixeles_10x']), int(anotacion['y_pixeles_10x']), int(anotacion['radio_pixeles_10x'])]]
    contenedor_esquinas = get_esquinas_parches_v2(lado_parche, anotacion_pixeles)
    print(contenedor_esquinas)
    comandos_ndpi = get_comandos_terminal(contenedor_esquinas, lado_parche, anotacion['Nombre slide'],id_anotacion=anotacion['ID anotación'],clase_anotacion=anotacion['Tipo'])
    print(comandos_ndpi)
    
    for comando in comandos_ndpi:
        call(comando, shell=True)
        comando_splited = comando.split('_')
        nombre_slide =anotacion['Nombre slide'].split('.')[0]
        path_recorte = path_carpeta_base +nombre_slide+'_x10_z0_idparche_'+comando_splited[1]+'_clas_'+abrebiar_clases(anotacion['Tipo'])+'.tif'
        
        print('=============================================================================')
        img = cv2.imread(path_recorte)
        prop_tejido = get_proporcion_tejido(img)
        print(prop_tejido)
        if prop_tejido<0.3:
            os.remove(path_recorte)

    #calcular el porcentaje de tenido de la imagen cortada
    #eliminarla si es mejor a 70%
    
    
print('cortado finalizado')


In [ ]:
def get_proporcion_tejido(image):
    """Calcula la proporción de tejido que hay en una imagen con respecto al
    total de pixeles en la imagen.
    Para realizar esto, primero se calcula la cantidad de pixeles grises (
    aquellos que no contienen tejido) usando simplemente un inRange en la
    imagen RGB. Luego, se calcula la diferencia (es decir, los pixeles que
    sí tienen tejido) y con ello se calcula su proporción respecto al total de
    pixeles.
    Args:
        - image: np.array(), imagen con tres canales
    Returns:
        float en el rango [0,1] que representa la proporción de tejido en la
        imagen con respecto al total de pixeles en ésta.
    """
    lower = np.array([225, 225, 225])
    upper = np.array([255, 255, 255])

    non_tissue_pixels = cv2.inRange(image, lower, upper)
    n_non_tissue_pixels = np.count_nonzero(non_tissue_pixels)
    n_pixels = image.shape[0] * image.shape[1]
    n_tissue_pixels = n_pixels - n_non_tissue_pixels

    return n_tissue_pixels / n_pixels


In [ ]:

path_recorte= cv2.imread('/Users/luisescares/Desktop/creacion-db-gruesa/anotacion1/1_x10_z0_idparche_a15f7c0_clas_rld.tif')
prop_tejido = get_proporcion_tejido(path_recorte)
print(prop_tejido)
        

## -Split db tejido-tumoral/tejido-no-tumoral (24 febrero)

In [ ]:
import shutil
import os

path_carpeta_recortes='/Users/luisescares/Desktop/creacion-db-gruesa/recortes-6clases'
if os.path.exists(path_carpeta_recortes+'/.DS_Store'):
    os.remove(path_carpeta_recortes+'/.DS_Store')
    
#crear carpeta db-tejidotumoral-tejidonotumoral si es que no existe
if os.path.exists('/Users/luisescares/Desktop/creacion-db-gruesa/db-tejidotumoral-tejidonotumoral'):
    os.remove('/Users/luisescares/Desktop/creacion-db-gruesa/db-tejidotumoral-tejidonotumoral')
path_tejido_no_tumoral='/Users/luisescares/Desktop/creacion-db-gruesa/db-tejidotumoral-tejidonotumoral/tejidonotumoral'
if not os.path.exists(path_tejido_no_tumoral):
    os.mkdir(path_tejido_no_tumoral)    
path_tejido_tumoral='/Users/luisescares/Desktop/creacion-db-gruesa/db-tejidotumoral-tejidonotumoral/tejidotumoral'
if not os.path.exists(path_tejido_tumoral):
    os.mkdir(path_tejido_tumoral)    

paths_recortes = os.listdir(path_carpeta_recortes)
def get_class(nombre_recorte):
    return nombre_recorte.split('clas_')[1].split('.')[0]

for nombre_recorte in paths_recortes:
    clase=get_class(nombre_recorte)
    if clase=='nt':
        shutil.move(path_carpeta_recortes+'/'+nombre_recorte, path_tejido_no_tumoral+'/'+nombre_recorte)
    if clase=='sr' or clase=='rnl' or clase=='rlci' or clase=='rld' or clase=='rlf':
        shutil.move(path_carpeta_recortes+'/'+nombre_recorte, path_tejido_tumoral+'/'+nombre_recorte)
print('imagenes reubicadas correctamente')  

## -Split db Tejido tumoral 5 clases(25 febrero)

In [ ]:
import shutil
import os
path_carpeta_recortes='/Users/luisescares/Desktop/creacion-db-gruesa/recortes-6clases'
if os.path.exists(path_carpeta_recortes+'/.DS_Store'):
    os.remove(path_carpeta_recortes+'/.DS_Store')
#crear carpeta db-5clases si es que no existe
if not os.path.exists('/Users/luisescares/Desktop/creacion-db-gruesa/db-5clases'):
    os.mkdir('/Users/luisescares/Desktop/creacion-db-gruesa/db-5clases')    

    
path_sr='/Users/luisescares/Desktop/creacion-db-gruesa/db-5clases/sr'
if not os.path.exists(path_sr):
    os.mkdir(path_sr)    
path_rnl='/Users/luisescares/Desktop/creacion-db-gruesa/db-5clases/rnl'
if not os.path.exists(path_rnl):
    os.mkdir(path_rnl)    
path_rlci='/Users/luisescares/Desktop/creacion-db-gruesa/db-5clases/rlci'
if not os.path.exists(path_rlci):
    os.mkdir(path_rlci)    
path_rld='/Users/luisescares/Desktop/creacion-db-gruesa/db-5clases/rld'
if not os.path.exists(path_rld):
    os.mkdir(path_rld)    
path_rlf='/Users/luisescares/Desktop/creacion-db-gruesa/db-5clases/rlf'
if not os.path.exists(path_rlf):
    os.mkdir(path_rlf)    


paths_recortes = os.listdir(path_carpeta_recortes)


def get_class(nombre_recorte):
    return nombre_recorte.split('clas_')[1].split('.')[0]

for nombre_recorte in paths_recortes:
    clase=get_class(nombre_recorte)
    if clase=='sr':
        shutil.move(path_carpeta_recortes+'/'+nombre_recorte, path_sr+'/'+nombre_recorte)
    if clase=='rnl':
        shutil.move(path_carpeta_recortes+'/'+nombre_recorte, path_rnl+'/'+nombre_recorte)
    if clase=='rlci':
        shutil.move(path_carpeta_recortes+'/'+nombre_recorte, path_rlci+'/'+nombre_recorte)
    if clase=='rld':
        shutil.move(path_carpeta_recortes+'/'+nombre_recorte, path_rld+'/'+nombre_recorte)
    if clase=='rlf':
        shutil.move(path_carpeta_recortes+'/'+nombre_recorte, path_rlf+'/'+nombre_recorte)
print('imagenes reubicadas correctamente')

#Celdas para verificar que se calculan bien las esquinas

In [ ]:
path_excel_infoHER2_luis='/Users/luisescares/Desktop/creacion-db-gruesa/her2_luis.xlsx'
anotaciones_excel = pd.read_excel(path_excel_infoHER2_luis)

In [ ]:
def draw_grid(img,contenedor_esquinas, lado_parche):    
    ID_anotacion = 0
    for esquinas in contenedor_esquinas:
        ID_fila = 0
        for fila in esquinas:
            ID_col = 0
            for esquina in fila:
                x_sup_izq = int(esquina[0])
                y_sup_izq = int(esquina[1])
                pto_inicial = (x_sup_izq, y_sup_izq)
                x_inf_der = x_sup_izq+lado_parche
                y_inf_der = y_sup_izq+lado_parche
                pto_final =  (x_inf_der, y_inf_der)
                color = (0,255,0)
                thickness = int(0.5)
                img = cv2.rectangle(img, pto_inicial, pto_final, color, thickness)
                ID_parche = 'a'+str(ID_anotacion) + 'f'+str(ID_fila) + 'c'+str(ID_col)                
                pos_ID_parche = (int(x_sup_izq + lado_parche/2) , y_sup_izq)
                font = cv2.FONT_HERSHEY_SIMPLEX 
                fontScale = int(1)
                thickness = 1# 1 pixel
                #cv2.putText(img,ID_parche, pos_ID_parche, font, fontScale, color, thickness, cv2.LINE_AA,False)                 
                ID_col +=1
            ID_fila +=1
        ID_anotacion +=1
    return img


In [ ]:
lado_parche=60
contenedor_esquinas = [
                        [
                            [[15090, 1527]], 
                            [[15056, 1557], [15086, 1557], [15116, 1557]],
                            [[15080, 1587], [15110, 1587], [15140, 1587]], 
                            [[15106, 1617]]
                        ]
                       ]



path_img='/Users/luisescares/Desktop/creacion-db-gruesa/1_x5_z0.tif'
img = cv2.imread(path_img)
#las esquinas y el lado parche estan adaptados a la escala x10
img = draw_grid(img,contenedor_esquinas, lado_parche)

centro_roi=(int(30226*0.5), int(3174*0.5))


img = cv2.circle(img, centro_roi,64, color=(255,0,0))




print(' alto es img.shape[0] , coordenada Y= ',img.shape[0], '        ancho es img.shape[1], coordenada X = ',img.shape[1])
folder_path='/Users/luisescares/Desktop/creacion-db-gruesa/'
cv2.imwrite(folder_path+'imagen_1_malla_dibujada.tif',img)


In [ ]:
path_img='/Users/luisescares/Desktop/creacion-db-gruesa/1_x5_z0.tif'
img = cv2.imread(path_img)
print(img)

In [ ]:
'''era para listar los nombres de archivps
import os
path_carpeta_ndpi_images='/Users/luisescares/Desktop/creacion-db-gruesa/archivos-ndpi'
paths_imagenes_ndpi = os.listdir(path_carpeta_ndpi_images)
paths_imagenes_ndpi.remove('.DS_Store')
a=[[path.split('-')[2]+'.ndpi',path] for path in paths_imagenes_ndpi]
'''